In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
from qdrant_client import QdrantClient, models 

qd_client = QdrantClient('http://localhost:6333')
EMBEDDING_DIMENSIONALITY = 512 
model_handle = 'jinaai/jina-embeddings-v2-small-en'

In [ ]:
collection_name = 'zoomcamp-faq' 
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE,
    )
)

True

In [16]:
# create index for filtering.
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name='course',
    field_schema='keyword',
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [7]:
points = []
for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
  
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc,
    )
    points.append(point)

In [9]:
qd_client.upsert(
    collection_name=collection_name,
    points=points,
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [17]:
query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        model=model_handle,
        text='I just discovered the course. can i still join it?',
    ),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key='course',
                match=models.MatchValue(value='data-engineering-zoomcamp'),
            )
        ]
    ),
    limit=5,
    with_payload=True,
)
query_points.points

[ScoredPoint(id=2, version=0, score=0.8603326, payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=7, version=0, score=0.8574545, payload={'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}, vecto

In [18]:
def vector_search(question):
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            model=model_handle,
            text=question,
        ),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key='course',
                    match=models.MatchValue(value='data-engineering-zoomcamp'),
                )
            ]
        ),
        limit=5,
        with_payload=True,
    )
    results = []
    for point in query_points.points:
        results.append(point.payload)
    return results

In [19]:
vector_search('I just discovered the course. can I still join it?')

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [20]:
from openai import OpenAI

client = OpenAI()

In [21]:
def search(query):


    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [23]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [25]:
rag('how do I run kafka?')

NameError: name 'index' is not defined

In [18]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even if it has already started. You are also eligible to submit the homework assignments. However, make sure to pay attention to the deadlines for turning in the final projects to avoid leaving everything until the last minute.'